In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import statistics
import plotly.express as px

# Análise Exploratória de Dados Imobiliários

## 01. Conjunto de Dados : Contexto


Os dados utilizados nesta análise foram coletados de um portal de anúncios de imóveis brasileiro.
Foram removidos dados sensíveis como o número de contato dos proprietários e a descrição textual do anúncio, de forma que não fosse possível identificá-los.

Os dados descrevem características estruturais como, área construída (usableAreas), número de suítes, banheiros, quartos, vagas de estacionamento. E características qualitativas como localização e facilidades (amenities). Além de apresentar a modalidade de negócio e precificação.

**Motivação:** A escolha por analisar dados imobiliários sobreveio por o mesmo ser frequentemente estudado no mercado de Dados, para o desenvolvimento de **Sistemas de Recomendação** e **Predição de Preço de Imóveis**. O primeiro para melhorar a jornada de compra, oferecendo aos clientes imóveis que melhor atendam à seus gostos; e o segundo para que empresas saibam o melhor momento de aquisição (melhor preço).

Nesta análise, nosso objeto de estudo é a variável de **preço de aluguel** em função dos demais descritores.

Algumas hipóteses a serem testadas:
- Casas em bairros centrais apresentaram alugueis mais custosos.
- As facilidades podem influenciar no encarecimento do aluguel.
- Casas com maiores areas construidas possivelmente apresentam alugueís mais caros.

In [2]:
pd.set_option('display.max_columns', 30)

In [3]:
df = pd.read_csv('dados_imoveis_sp.csv')

df.head()

,amenities,usableAreas,id,parkingSpaces,address,suites,bathrooms,totalAreas,bedrooms,pricingInfos
0,"['PETS_ALLOWED', 'ELEVATOR', 'GARDEN', 'ELECTR...",['101'],2574084550,[1],"{'country': 'BR', 'zipCode': '04734003', 'geoJ...",[],[2],['111'],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
1,"['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...",['140'],2583748663,[2],"{'country': 'BR', 'zipCode': '01307000', 'geoJ...",[2],[4],[],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
2,"['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...",['50'],2562971980,[1],"{'country': 'BR', 'zipCode': '01209010', 'geoJ...",[0],[1],['50'],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
3,"['POOL', 'BARBECUE_GRILL', 'GATED_COMMUNITY', ...",['58'],2580478200,[1],"{'country': 'BR', 'zipCode': '01127000', 'geoJ...",[],[1],[],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
4,"['PETS_ALLOWED', 'GATED_COMMUNITY', 'ELECTRONI...",['64'],2583729583,[1],"{'country': 'BR', 'zipCode': '05435001', 'geoJ...",[],[1],['80'],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."


In [4]:
linhas,colunas = df.shape
print(f'{linhas} instâncias e {colunas} atributos')

10000 instâncias e 10 atributos


## 02. Limpeza e Tratamento de dados

### 2.1 Remoção de dados duplicados

In [5]:
print('Elementos repetidos pelo atributo ID:',linhas - df['id'].unique().shape[0])

Elementos repetidos pelo atributo ID: 72


In [6]:
print('Elementos repetidos pelo método duplicated:', df[df.duplicated()].shape[0])

Elementos repetidos pelo método duplicated: 71


In [7]:
# remove os duplicados
# Only consider certain columns for identifying duplicates, by default use all of the columns.
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html
df.drop_duplicates(inplace=True)

In [8]:
linhas,colunas = df.shape
print(f'{linhas} instâncias e {colunas} atributos')

9929 instâncias e 10 atributos


In [9]:
print('Elementos repetidos pelo atributo ID:',linhas - df['id'].unique().shape[0])

Elementos repetidos pelo atributo ID: 1


In [10]:
id_duplicado = int(df[df['id'].duplicated()]['id'])
duplicados = df[df.id == id_duplicado]
duplicados

,amenities,usableAreas,id,parkingSpaces,address,suites,bathrooms,totalAreas,bedrooms,pricingInfos
6707,"['POOL', 'BARBECUE_GRILL', 'ELEVATOR', 'GATED_...",['92'],2583627481,[2],"{'country': 'BR', 'zipCode': '01153000', 'geoJ...",[1],[3],['92'],[3],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
7099,"['POOL', 'BARBECUE_GRILL', 'ELEVATOR', 'GATED_...",['92'],2583627481,[2],"{'country': 'BR', 'zipCode': '01153000', 'geoJ...",[1],[3],['92'],[3],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."


In [11]:
print('Possuem mesmo pricingInfos?',duplicados.pricingInfos[6707] == duplicados.pricingInfos[7099])

Possuem mesmo pricingInfos? False


### 2.2 Parseando dados

Com exceção da coluna **id**, todos os atributos deste dataset estão no formato **string**. 

É possível notar esses atributos contém **literais** de estruturas de dados como **listas e dicionários**.

Inicialmente, uma conversão para seu real tipo de dado facilitaria a extração e tratamento de seus valores, uma vez que teríamos uma estrutura de dados usável.

In [12]:
for atributo,valor in zip(list(df),list(df.iloc[0])):
    print(f'{atributo}: {valor}, tipo:{type(valor)}\n')

amenities: ['PETS_ALLOWED', 'ELEVATOR', 'GARDEN', 'ELECTRONIC_GATE', 'CONCIERGE_24H'], tipo:<class 'str'>

usableAreas: ['101'], tipo:<class 'str'>

id: 2574084550, tipo:<class 'numpy.int64'>

parkingSpaces: [1], tipo:<class 'str'>

address: {'country': 'BR', 'zipCode': '04734003', 'geoJson': '', 'city': 'São Paulo', 'streetNumber': '1850', 'level': 'STREET', 'precision': 'ROOFTOP', 'confidence': 'VALID_STREET', 'stateAcronym': 'SP', 'source': 'CORREIOS', 'point': {'lon': -46.695829, 'source': 'GOOGLE', 'lat': -23.638282}, 'ibgeCityId': '', 'zone': 'Zona Sul', 'street': 'Avenida Adolfo Pinheiro', 'locationId': 'BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Santo Amaro', 'district': '', 'name': '', 'state': 'São Paulo', 'neighborhood': 'Santo Amaro', 'poisList': ['BS:Graham Bell C/B', 'BS:Graham Bell B/C', 'BS:Rua Verbo Divino, 61', 'BS:Américo Brasiliense C/B', 'BS:Parada Marechal Deodoro 2 - B/C', 'TS:Graham Bell C/B', 'TS:Graham Bell B/C', 'TS:Rua Verbo Divino, 61', 'TS:Américo Brasiliense C/

O comando **literal_eval** é um interessante comando da biblioteca **ast – Abstract Syntax Trees**. 

Ele converte(parsing) os recursos *stringificados* em seus objetos python correspondentes

In [13]:
# trazendo coluna "id" para o ínicio
cols = list(df.columns)
cols.remove('id')
cols.insert(0,'id')
df = df[cols]

# convertendo as demais colunas em estruturas de dados usáveis.
for col in cols[1:]:
    df[col] = df[col].apply(literal_eval)

In [14]:
# Nas colunas abaixo, seus valores estão dentro de listas
cols = ['usableAreas','parkingSpaces','suites','bathrooms','totalAreas','bedrooms']

# Simplesmente, extraímos seu conteúdo, acessando a primeira posiçao e substituindo por NaN quando vazio.
for var in cols:
    df[var] = df[var].apply(lambda x: float(x[0]) if x else np.nan)
    
df.head()

,id,amenities,usableAreas,parkingSpaces,address,suites,bathrooms,totalAreas,bedrooms,pricingInfos
0,2574084550,"[PETS_ALLOWED, ELEVATOR, GARDEN, ELECTRONIC_GA...",101.0,1.0,"{'country': 'BR', 'zipCode': '04734003', 'geoJ...",NaN,2.0,111.0,2.0,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
1,2583748663,"[POOL, FURNISHED, BARBECUE_GRILL, ELEVATOR, GY...",140.0,2.0,"{'country': 'BR', 'zipCode': '01307000', 'geoJ...",2.0,4.0,NaN,2.0,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
2,2562971980,"[POOL, FURNISHED, BARBECUE_GRILL, ELEVATOR, GA...",50.0,1.0,"{'country': 'BR', 'zipCode': '01209010', 'geoJ...",0.0,1.0,50.0,2.0,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
3,2580478200,"[POOL, BARBECUE_GRILL, GATED_COMMUNITY, GYM, G...",58.0,1.0,"{'country': 'BR', 'zipCode': '01127000', 'geoJ...",NaN,1.0,NaN,2.0,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
4,2583729583,"[PETS_ALLOWED, GATED_COMMUNITY, ELECTRONIC_GAT...",64.0,1.0,"{'country': 'BR', 'zipCode': '05435001', 'geoJ...",NaN,1.0,80.0,2.0,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."


### 2.3 Tratando dados categóricos

A coluna *'amenities'* apresenta as facilidades que cada imóvel pode oferecer.
- Como podemos aproveitar essas informações?

In [15]:
df['amenities'][:5]

0    [PETS_ALLOWED, ELEVATOR, GARDEN, ELECTRONIC_GA...
1    [POOL, FURNISHED, BARBECUE_GRILL, ELEVATOR, GY...
2    [POOL, FURNISHED, BARBECUE_GRILL, ELEVATOR, GA...
3    [POOL, BARBECUE_GRILL, GATED_COMMUNITY, GYM, G...
4    [PETS_ALLOWED, GATED_COMMUNITY, ELECTRONIC_GAT...
Name: amenities, dtype: object

In [16]:
print('Top 10 Facilidades dos Imóveis de São Paulo:')
print(df['amenities'].explode().value_counts()[:10])

Top 10 Facilidades dos Imóveis de São Paulo:
ELEVATOR           5101
POOL               4583
PARTY_HALL         4088
BARBECUE_GRILL     3938
SERVICE_AREA       3701
GYM                3667
PLAYGROUND         3336
GARDEN             3184
INTERCOM           2882
GATED_COMMUNITY    2836
Name: amenities, dtype: int64


Decidimos utilizar as 10 principais facilidades e utilizar a técnica de codificação de dados categóricos, **One Hot Enconding**.

Para cada instância de atributo categórico, criamos uma nova variável. Cada categoria é mapeada com uma variável binária contendo 0 ou 1. Aqui, 0 representa a ausência e 1 representa a presença dessa categoria.

In [17]:
top10_amenities = list(df['amenities'].explode().value_counts()[:10].index)

In [18]:
def has_amenity(amenities,amenity):
    if amenity in amenities:
        return 1
    else:
        return 0
    
# Cria uma coluna para cada uma das 10 principais facilidades
# com valor binário, representando ausência ou ocorrência da mesma
for amenity in top10_amenities:
    df[amenity.lower()] = df['amenities'].apply(has_amenity,amenity=amenity)

In [19]:
df[[amenity.lower() for amenity in top10_amenities]].head()

,elevator,pool,party_hall,barbecue_grill,service_area,gym,playground,garden,intercom,gated_community
0,1,0,0,0,0,0,0,1,0,0
1,1,1,1,1,0,1,1,1,0,0
2,1,1,1,1,0,1,1,1,0,1
3,0,1,1,1,0,1,1,1,0,1
4,1,0,0,0,0,0,0,0,0,1


### 2.4 Extraindo informações relevantes

A coluna 'address' é dicionário que contém diversos atributos relacionados ao endereço do imóvel.

- Extrair o CEP, Bairro, Zona, Latitude e Longitude podem ser muito uteís.

In [20]:
df['address'][0]

{'country': 'BR',
 'zipCode': '04734003',
 'geoJson': '',
 'city': 'São Paulo',
 'streetNumber': '1850',
 'level': 'STREET',
 'precision': 'ROOFTOP',
 'confidence': 'VALID_STREET',
 'stateAcronym': 'SP',
 'source': 'CORREIOS',
 'point': {'lon': -46.695829, 'source': 'GOOGLE', 'lat': -23.638282},
 'ibgeCityId': '',
 'zone': 'Zona Sul',
 'street': 'Avenida Adolfo Pinheiro',
 'locationId': 'BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Santo Amaro',
 'district': '',
 'name': '',
 'state': 'São Paulo',
 'neighborhood': 'Santo Amaro',
 'poisList': ['BS:Graham Bell C/B',
  'BS:Graham Bell B/C',
  'BS:Rua Verbo Divino, 61',
  'BS:Américo Brasiliense C/B',
  'BS:Parada Marechal Deodoro 2 - B/C',
  'TS:Graham Bell C/B',
  'TS:Graham Bell B/C',
  'TS:Rua Verbo Divino, 61',
  'TS:Américo Brasiliense C/B',
  'TS:Parada Marechal Deodoro 2 - B/C',
  'CS:7 Molinos',
  'CS:Casa de Pães Neblina Paulista',
  'CS:Casa de Bolo',
  'CS:Berna',
  'CS:Gêmel',
  'VP:Kennel Club'],
 'pois': [],
 'valuableZones': [{'cit

A coluna 'pricingInfos' pode possuir dois dicionários. Um contendo preço de aluguel, e outro com preço de compra. Estamos interessados em pegar apenas o preço de aluguel.

In [21]:
df['pricingInfos'][91]

[{'rentalInfo': {'period': 'MONTHLY',
   'warranties': ['INSURANCE_GUARANTEE', 'GUARANTOR']},
  'yearlyIptu': '120',
  'price': '410000',
  'businessType': 'SALE',
  'monthlyCondoFee': '700'},
 {'rentalInfo': {'period': 'MONTHLY',
   'warranties': ['INSURANCE_GUARANTEE', 'GUARANTOR'],
   'monthlyRentalTotalPrice': '2400'},
  'yearlyIptu': '120',
  'price': '1700',
  'businessType': 'RENTAL',
  'monthlyCondoFee': '700'}]

In [22]:
def extract_neighborhood(address):
    neighborhood = address['neighborhood']
    
    return neighborhood

In [23]:
def extract_zone(address):
    zone = address['zone']
    
    return zone

In [24]:
def extract_zipcode(address):
    zipCode = address['zipCode']
    
    return zipCode

In [25]:
def extract_latitude(address):
    try:
        latitude = address['point']['lat']
    except:
        latitude = np.nan
        
    return latitude

In [26]:
def extract_longitude(address):
    try:
        longitude = address['point']['lon']
    except:
        longitude = np.nan
        
    return longitude

In [27]:
def get_rental_price(pricingInfos):
    price = [info['price'] for info\
     in pricingInfos\
     if info['businessType'] == 'RENTAL'][0]
    
    return float(price)

In [28]:
# extraindo as informações relevantes
df['zipCode'] = df['address'].apply(extract_zipcode)
df['zone'] = df['address'].apply(extract_zone)
df['neighborhood'] = df['address'].apply(extract_neighborhood)
df['latitude'] = df['address'].apply(extract_latitude)
df['longitude'] = df['address'].apply(extract_longitude)
df['rental_price'] = df['pricingInfos'].apply(get_rental_price)

In [29]:
df[['zipCode','zone','neighborhood','latitude','longitude','rental_price']].head()

,zipCode,zone,neighborhood,latitude,longitude,rental_price
0,04734003,Zona Sul,Santo Amaro,-23.638282,-46.695829,2300.0
1,01307000,Centro,Consolação,-23.553415,-46.653839,9500.0
2,01209010,Centro,Santa Efigênia,-23.540638,-46.642793,3000.0
3,01127000,Centro,Bom Retiro,NaN,NaN,1900.0
4,05435001,Zona Oeste,Sumarezinho,-23.550271,-46.691135,2400.0


### 2.5 Eliminando colunas desnecessárias

In [30]:
# elimina as colunas 
df.drop(['id','totalAreas','amenities','address','pricingInfos'], axis=1,inplace=True)

In [31]:
df.head()

,usableAreas,parkingSpaces,suites,bathrooms,bedrooms,elevator,pool,party_hall,barbecue_grill,service_area,gym,playground,garden,intercom,gated_community,zipCode,zone,neighborhood,latitude,longitude,rental_price
0,101.0,1.0,NaN,2.0,2.0,1,0,0,0,0,0,0,1,0,0,04734003,Zona Sul,Santo Amaro,-23.638282,-46.695829,2300.0
1,140.0,2.0,2.0,4.0,2.0,1,1,1,1,0,1,1,1,0,0,01307000,Centro,Consolação,-23.553415,-46.653839,9500.0
2,50.0,1.0,0.0,1.0,2.0,1,1,1,1,0,1,1,1,0,1,01209010,Centro,Santa Efigênia,-23.540638,-46.642793,3000.0
3,58.0,1.0,NaN,1.0,2.0,0,1,1,1,0,1,1,1,0,1,01127000,Centro,Bom Retiro,NaN,NaN,1900.0
4,64.0,1.0,NaN,1.0,2.0,1,0,0,0,0,0,0,0,0,1,05435001,Zona Oeste,Sumarezinho,-23.550271,-46.691135,2400.0


### 2.6 Tratamento de Outliers

Identificando outliers e substituindo por um NaN. Assumimos, por simplicidade, que todos outliers são, na verdade, dados faltantes. Fundamento teórico: https://www.scribbr.com/statistics/outliers/

In [32]:
cols = ['usableAreas','parkingSpaces','suites','bathrooms','bedrooms', 'rental_price']
for col in cols:
    median = df[col].median()
    Q1 = df[col].quantile(q=0.25)
    Q3 = df[col].quantile(q=0.75)
    IQ = Q3-Q1
    lim_sup = Q3+1.5*IQ
    lim_inf = Q1-1.5*IQ

    df[col]=np.where(((df[col]<lim_inf)|(df[col]>lim_sup)),np.nan,df[col])

### 2.7 Tratamento de valores faltantes

Pelas informações do dataframe, existem 2 colunas com dados faltantes que precisam ser tratados.

In [33]:
df.columns[df.isna().any()].tolist()

['usableAreas',
 'parkingSpaces',
 'suites',
 'bathrooms',
 'bedrooms',
 'latitude',
 'longitude',
 'rental_price']

Desconsidera-se latitude e longitude pois não se aplicam neste tratamento.

In [34]:
(df.isnull().sum() / df.shape[0])*100

usableAreas         7.795347
parkingSpaces       7.311915
suites             22.862322
bathrooms          11.199517
bedrooms            0.010072
elevator            0.000000
pool                0.000000
party_hall          0.000000
barbecue_grill      0.000000
service_area        0.000000
gym                 0.000000
playground          0.000000
garden              0.000000
intercom            0.000000
gated_community     0.000000
zipCode             0.000000
zone                0.000000
neighborhood        0.000000
latitude           47.577802
longitude          47.577802
rental_price        9.265787
dtype: float64

Separando as colunas que contem dados faltantes em um dataframe

In [35]:
columns_with_nan = ['usableAreas','parkingSpaces','suites','bathrooms','bedrooms','rental_price']
replace_by = ['median','mode','mode','mode','mode','median']

for col,rpl_by in zip(columns_with_nan,replace_by):
    replacement_function = {
        'median':statistics.median,
        'mode':statistics.mode
    }
    df[col].replace(np.NaN, replacement_function[rpl_by](df[col]),inplace=True)

Atribuindo colunas aos seus respectivos tipos

In [36]:
df = df.astype({'parkingSpaces': 'int32','suites': 'int32','bathrooms': 'int32','bedrooms': 'int32','zone':'category','neighborhood':'category'})

In [37]:
df.dtypes

usableAreas         float64
parkingSpaces         int32
suites                int32
bathrooms             int32
bedrooms              int32
elevator              int64
pool                  int64
party_hall            int64
barbecue_grill        int64
service_area          int64
gym                   int64
playground            int64
garden                int64
intercom              int64
gated_community       int64
zipCode              object
zone               category
neighborhood       category
latitude            float64
longitude           float64
rental_price        float64
dtype: object

## 03. Análise dos dados

### 3.1 Análise univariada

In [38]:
df.describe()

,usableAreas,parkingSpaces,suites,bathrooms,bedrooms,elevator,pool,party_hall,barbecue_grill,service_area,gym,playground,garden,intercom,gated_community,latitude,longitude,rental_price
count,9929.000000,9929.000000,9929.000000,9929.000000,9929.000000,9929.000000,9929.000000,9929.000000,9929.000000,9929.000000,9929.000000,9929.000000,9929.000000,9929.000000,9929.000000,5205.000000,5205.000000,9929.000000
mean,71.845805,1.185618,0.705912,1.524423,2.082788,0.513748,0.461577,0.411723,0.396616,0.372747,0.369322,0.335985,0.320677,0.290261,0.285628,-23.577059,-46.656474,2976.346863
std,30.852919,0.731122,0.567313,0.686899,0.837764,0.499836,0.498547,0.492170,0.489220,0.483560,0.482646,0.472358,0.466760,0.453906,0.451736,0.044009,0.052310,1601.093785
min,10.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-23.774967,-46.803816,500.000000
25%,49.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-23.608538,-46.685559,1800.000000
50%,65.000000,1.000000,1.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-23.572972,-46.660505,2650.000000
75%,95.000000,2.000000,1.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-23.545965,-46.635038,3600.000000
max,166.000000,3.000000,2.000000,3.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-23.437146,-46.376025,8540.000000


In [39]:
rental_price_mean = round(df["rental_price"].mean(),2)
print(f'A média do aluguel dos imóveis em São Paulo: R$ {rental_price_mean}.')

A média do aluguel dos imóveis em São Paulo: R$ 2976.35.


In [40]:
df['neighborhood'].value_counts().sort_values(ascending=False)

Vila Mariana       404
Pinheiros          362
Bela Vista         317
Indianópolis       293
Jardim Paulista    281
                  ... 
Vila Genioli         1
Vila Guaca           1
Vila do Bosque       1
Vila Industrial      1
Mandaqui             1
Name: neighborhood, Length: 600, dtype: int64

In [41]:
df['zone'].value_counts().sort_values(ascending=False)

Zona Sul      3620
Zona Oeste    2993
Centro        1496
Zona Leste    1171
Zona Norte     647
                 2
Name: zone, dtype: int64

### Quantos imóveis estão abaixo e acima da média (aluguel) ?

In [42]:
def above_average(rental_price):
    if rental_price > rental_price_mean:
        return 1
    else:
        return 0

In [43]:
df['above_average'] = df['rental_price'].apply(above_average)

In [44]:
abaixo, acima = list(df['above_average'].value_counts(normalize=True)*100)
print(f'{abaixo:.2f}% dos imóveis estão abaixo do preço médio')
print(f'E {acima:.2f}% dos imóveis estão acima do preço médio')

63.47% dos imóveis estão abaixo do preço médio
E 36.53% dos imóveis estão acima do preço médio


### 3.2 Análise bivariada

In [45]:
df.pivot_table(index='neighborhood', values='rental_price', aggfunc='mean').sort_values(by='rental_price',ascending=False)

,rental_price
neighborhood,
Bela Aliança,6108.500000
Granja Julieta,5750.000000
Jurubatuba,5033.333333
Vila do Sol,5000.000000
Jardim Guarapiranga,5000.000000
...,...
Cidade Tiradentes,727.500000
Vila Nova Curuçá,700.000000
Jardim Jaraguá,700.000000


In [46]:
df.pivot_table(index='zone', values='rental_price', aggfunc='mean').sort_values(by='rental_price',ascending=False)

,rental_price
zone,
Zona Oeste,3697.705312
Zona Sul,3028.837293
Centro,2556.797460
Zona Leste,2083.771136
,2027.500000
Zona Norte,1934.165379


In [47]:
amenities_values = dict()
for amenity in top10_amenities:
    amenity = amenity.lower()
    with_amenity = df[df[amenity]==1]["rental_price"].mean()
    without_amenity = df[df[amenity]==0]["rental_price"].mean()
    amenities_values[amenity] = [with_amenity,without_amenity,with_amenity-without_amenity]

ordered_amenities_values = {amenities:prices for amenities,prices in sorted(amenities_values.items(), key=lambda x: x[1][2], reverse=True)}


In [48]:
for amenity,prices in ordered_amenities_values.items():
    print(f'Média de valores de imóveis com {amenity}: R$ {prices[0]:.2f}.')
    print(f'Média de valores de imóveis sem {amenity}: R$ {prices[1]:.2f}.')
    print(f'Diferença: R$ {prices[2]:.2f}.')
    print('-'*30)

Média de valores de imóveis com pool: R$ 3439.54.
Média de valores de imóveis sem pool: R$ 2579.27.
Diferença: R$ 860.27.
------------------------------
Média de valores de imóveis com gym: R$ 3437.37.
Média de valores de imóveis sem gym: R$ 2706.37.
Diferença: R$ 731.00.
------------------------------
Média de valores de imóveis com playground: R$ 3399.18.
Média de valores de imóveis sem playground: R$ 2762.40.
Diferença: R$ 636.78.
------------------------------
Média de valores de imóveis com party_hall: R$ 3346.81.
Média de valores de imóveis sem party_hall: R$ 2717.07.
Diferença: R$ 629.74.
------------------------------
Média de valores de imóveis com garden: R$ 3378.62.
Média de valores de imóveis sem garden: R$ 2786.45.
Diferença: R$ 592.17.
------------------------------
Média de valores de imóveis com barbecue_grill: R$ 3219.11.
Média de valores de imóveis sem barbecue_grill: R$ 2816.77.
Diferença: R$ 402.34.
------------------------------
Média de valores de imóveis com elev

### Análise de correlação

In [49]:
cols = list(df.columns)
cols.remove('latitude')
cols.remove('longitude')

feat_corr = [(feature,corr) for feature, corr in zip(df[cols].corr()['rental_price'].index,df[cols].corr()['rental_price'])]
feat_corr.pop()

feat_corr = sorted(feat_corr, key=lambda x: x[1], reverse=True)

for feat, corr in feat_corr:
    print(f'Atributo: {feat} | Grau de Correlação: {corr:.2f}')

Atributo: rental_price | Grau de Correlação: 1.00
Atributo: usableAreas | Grau de Correlação: 0.51
Atributo: parkingSpaces | Grau de Correlação: 0.44
Atributo: suites | Grau de Correlação: 0.41
Atributo: bathrooms | Grau de Correlação: 0.34
Atributo: bedrooms | Grau de Correlação: 0.31
Atributo: pool | Grau de Correlação: 0.27
Atributo: gym | Grau de Correlação: 0.22
Atributo: party_hall | Grau de Correlação: 0.19
Atributo: playground | Grau de Correlação: 0.19
Atributo: garden | Grau de Correlação: 0.17
Atributo: barbecue_grill | Grau de Correlação: 0.12
Atributo: elevator | Grau de Correlação: 0.11
Atributo: service_area | Grau de Correlação: 0.08
Atributo: gated_community | Grau de Correlação: 0.07
Atributo: intercom | Grau de Correlação: 0.03


### 3.3 Visualizando imóveis acima e abaixo da média (aluguel)

In [50]:
imoveis_com_geoloc = df.query("latitude.notna()",engine='python')
imoveis_com_geoloc.shape

(5205, 22)

In [53]:
#color_scale = [(0, 'blue'), (1,'orange')]

fig = px.scatter_mapbox(imoveis_com_geoloc, 
                        lat="latitude", 
                        lon="longitude", 
                        hover_data=["rental_price","usableAreas"],
                        color="above_average",
                        color_continuous_scale=px.colors.sequential.Cividis_r,
                        size="rental_price",
                        zoom=8, 
                        height=800,
                        width=1200)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()